In [1]:
from geoimages import etl
from geoimages import neural
from keras.utils import plot_model
import h5py
import numpy as np

Using TensorFlow backend.


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Training Data

In [7]:
## read training data, normalize X, One-hot Y
labels = etl.Labels()
hf = h5py.File('../datasets/image_classification_train.h5', 'r')
X_Train_Orig = np.array(hf.get('X_Train_Orig'))
Y_Train_Orig = np.array(hf.get('Y_Train_Orig'))
X_Train = X_Train_Orig/255.
Y_Train_OH = labels.categorical_to_onehot(Y_Train_Orig)
hf.close()
print('X Train shape: ' + str(X_Train_Orig.shape))
print('Y Train shape: ' + str(Y_Train_Orig.shape))

X Train shape: (20998, 28, 28, 3)
Y Train shape: (20998,)


### Test Data

In [11]:
## read test data, normalize X, One-hot Y
hf = h5py.File('../datasets/image_classification_test.h5', 'r')
X_Test_Orig = np.array(hf.get('X_Test_Orig'))
Y_Test_Orig = np.array(hf.get('Y_Test_Orig'))
X_Test = X_Test_Orig/255.
Y_Test_OH = labels.categorical_to_onehot(Y_Test_Orig)
hf.close()
print('X Test shape: ' + str(X_Test_Orig.shape))
print('Y Test shape: ' + str(Y_Test_Orig.shape))

X Test shape: (3564, 28, 28, 3)
Y Test shape: (3564,)


### 1-Layer Convolutional Network

In [8]:
## create, compile, train model
model_1L = neural.Convolutional.Conv1L(X_Train.shape[1:])
model_1L.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1L.summary()
plot_model(model_1L, to_file='./assets/Conv1L.png')
model_1L.fit(x=X_Train, y=Y_Train_OH, batch_size=16, epochs=5, verbose=1)

Model: "Conv1L"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 32, 32, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 30, 30, 32)        896       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 30, 30, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7200)              0    

In [12]:
## evaluate test set
preds_1L = model_1L.evaluate(x=X_Test, y=Y_Test_OH)
print()
print ("Loss = " + str(preds_1L[0]))
print ("Test Accuracy = " + str(preds_1L[1]))

3564/3564 [==============================] - 4s 986us/step

Loss = 0.4817236965035081
Test Accuracy = 0.8751403093338013


In [31]:
## save model
model_1L.save('../models/model_1L.h5', include_optimizer=True)

### 2-Layer Convolutional Network

In [19]:
## create, compile, fit model
model_2L = neural.Convolutional.Conv2L(X_Train.shape[1:])
model_2L.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2L.summary()
plot_model(model_2L, to_file='./assets/Conv2L.png')
model_2L.fit(x=X_Train, y=Y_Train_OH, batch_size=16, epochs=5, verbose=1)

Model: "Conv2L"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 32, 32, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 30, 30, 32)        896       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 30, 30, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 15, 15, 32)        0    

In [27]:
## evaluate test set
preds_2L = model_2L.evaluate(x=X_Test, y=Y_Test_OH)
print()
print ("Loss = " + str(preds_2L[0]))
print ("Test Accuracy = " + str(preds_2L[1]))

3564/3564 [==============================] - 4s 1ms/step

Loss = 0.11371453629867262
Test Accuracy = 0.9635241031646729


In [33]:
## save model
model_2L.save('../models/model_2L.h5', include_optimizer=True)

### Small VGGNet

In [21]:
## create, compile, fit model
model_VGG = neural.Convolutional.smallVGGNet(X_Train.shape[1:])
model_VGG.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_VGG.summary()
plot_model(model_VGG, to_file='./assets/VGG.png')
model_VGG.fit(x=X_Train, y=Y_Train_OH, batch_size=16, epochs=5, verbose=1)

Model: "smallVGGNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 28, 28, 32)        896       
_________________________________________________________________
activation_10 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
bn0 (BatchNormalization)     (None, 28, 28, 32)        128       
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 9, 9, 32)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 9, 9, 32)          0         
_________________________________________________________________
conv1 (Conv2D)               (None, 9, 9, 64)          

In [25]:
## evaluate test set
preds_VGG = model_VGG.evaluate(x=X_Test, y=Y_Test_OH)
print()
print ("Loss = " + str(preds_VGG[0]))
print ("Test Accuracy = " + str(preds_VGG[1]))

3564/3564 [==============================] - 5s 2ms/step

Loss = 1.0302366168812067
Test Accuracy = 0.7314814925193787


In [34]:
## save model
model_VGG.save('../models/model_VGG.h5', include_optimizer=True)